# Experiment 2: Interactive reprocessing
This is representative of a new type of functionality that we currently don’t support in METASPACE
because it’s uneconomical with the serverful approach. While looking for specific compounds,
scientists tend to have relatively short lists of molecules of interest, and iteratively try
different adducts or modifiers until they find the data they’re interested in.

### METRICS TO BENCHMARK
* Performance:
    * **Metric:** Total processing time
    
        **Goal:** Fast enough to use interactively in a notebook - less than ~60 seconds

* Cost:
    * **Metric:** Total cost
    
        **Goal:** Significantly less than a full annotation - determined by experiment 1

# Notebook setup
Run `python3 setup.py install` to install all requirements for annotation pipeline project.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%config Completer.use_jedi = False
%matplotlib inline

In [ ]:
# We need this to overcome Python notebooks limitations of too many open files
import resource
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print('Before:', soft, hard)

# Raising the soft limit. Hard limits can be raised only by sudo users
resource.setrlimit(resource.RLIMIT_NOFILE, (10000, hard))
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print('After:', soft, hard)

In [ ]:
# These are Python and Python lib path we want to use
import sys
sys.executable, sys.prefix

In [ ]:
#Install PyWren-IBM if needed
try:
    import pywren_ibm_cloud as pywren
except ModuleNotFoundError:    
    !{sys.executable} -m pip install -U pywren-ibm-cloud==1.0.10
    import pywren_ibm_cloud as pywren

pywren.__version__

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

## Configuration

In [ ]:
import json
config = json.load(open('config.json'))

In [ ]:
#input_config = json.load(open('metabolomics/input_config_small.json'))
input_config = json.load(open('metabolomics/input_config_big.json'))
#input_config = json.load(open('metabolomics/input_config_huge.json'))
input_data = input_config['dataset']
input_db = input_config['molecular_db']

# Override databases, because this experiment expects a small database
exp_db_path = 'metabolomics/db/mol_db5.pickle'
input_db['databases'] = [exp_db_path]

# Initial setup (not included in benchmark timings)

### Load & segment dataset

In [ ]:
from annotation_pipeline.pipeline import Pipeline
pipeline = Pipeline(config, input_config)
pipeline.load_ds()
pipeline.split_ds()
pipeline.segment_ds()

# Benchmark

In [ ]:
from datetime import datetime
start_time = datetime.now()
print('start', start_time)

### Process new molecules

In [ ]:
from annotation_pipeline.molecular_db import build_database, calculate_centroids, get_formula_id_dfs, clean_formula_chunks

In [ ]:
# Upload list of molecules (in a real scenario this list would change every iteration, so this isn't part of setup)
import pandas as pd
mols = pd.read_csv('metabolomics/db/mol_db5.csv')
mols_list = sorted(set(mols.sf.values.tolist()))
cos_client.put_object(Bucket=config['storage']['db_bucket'], Key=exp_db_path, Body=pickle.dumps(mols_list))

In [ ]:
num_formulas, formula_chunk_keys = build_database(config, input_db)

In [ ]:
polarity = input_data['polarity']
isocalc_sigma = input_data['isocalc_sigma']
centroids_shape, centroids_head = calculate_centroids(config, input_db, formula_chunk_keys, polarity, isocalc_sigma)

In [ ]:
clean_formula_chunks(config, input_db, formula_chunk_keys)

In [ ]:
%time pipeline.segment_centroids()

### Run Annotation

In [ ]:
%time pipeline.annotate()

In [ ]:
%time results_df = pipeline.formula_metrics_df

In [ ]:
# Display results
print(results_df.shape)
results_df.head()

In [ ]:
finish_time = datetime.now()
print('start', start_time)
print('finish', finish_time)
print('duration', finish_time - start_time)

# Clean Temp Data

In [ ]:
from annotation_pipeline.utils import clean_from_cos

In [ ]:
# Clean dataset chunks
clean_from_cos(config, config["storage"]["ds_bucket"], input_data["ds_chunks"])

In [ ]:
# Clean dataset segments
clean_from_cos(config, config["storage"]["ds_bucket"], input_data["ds_segments"])

In [ ]:
# Clean centroids database segments
clean_from_cos(config, config["storage"]["db_bucket"], input_db["centroids_segments"])

In [ ]:
# Clean formula output images
clean_from_cos(config, config["storage"]["output_bucket"], output["formula_images"])

In [ ]:
# Clean FDR rankings
clean_from_cos(config, config["storage"]["ds_bucket"], input_data["fdr_rankings"])